# Notebook rozwiązauje prosty problem klasyfikacji gatunków irysów

Założeniem zadanie jest wytrenowanie modelów i wybranie najlepszego dla nas opcji przy użyciu usługi Azure Machine Learning, które mają za zadanie klasyfikować, który to gatunek irysa

Jak wspomniałam, użyjemy tutaj portalu Azure, więc początkowo posiadając już subskrypcję na stronie portal.azure.com należy postawić zasób Machine Learning, a następnie się do niego podłączyć. 

Poniżej pokazano możliwość podłączenia się do zasobu. Należy przy użyciu specjalnej biblioteki azureml zaimportować odpowiednie funkcje, dzięki którym połączymy sie z zasobem oraz danymi, które także są w chmurze - zasób Blob Storage. Należy takze podać subskrypcje, resource group (ponieważ każdy projekt musi mieć własną resource group) oraz nazwę naszego projektu.

In [3]:
# azureml-core of version 1.0.72 or higher is required
# azureml-dataprep[pandas] of version 1.1.34 or higher is required
from azureml.core import Workspace, Dataset

subscription_id = '03d60a7f-8a1f-4aa2-8e20-3f45e41abc75'
resource_group = 'aml-classification'
workspace_name = 'chmury-aml'

workspace = Workspace(subscription_id, resource_group, workspace_name)

dataset = Dataset.get_by_name(workspace, name='iris')
dataset.to_pandas_dataframe()

,petal_length,petal_width,sepal_length,sepal_width,label
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,Iris-virginica
146,6.3,2.5,5.0,1.9,Iris-virginica
147,6.5,3.0,5.2,2.0,Iris-virginica
148,6.2,3.4,5.4,2.3,Iris-virginica


Aby ocenić jakoś naszego modelu należy go przetestować, dlatego nasz zbiór podzielimy na dane uczace i testujące w proporcjach 70% - uczące, 30% - testujące. Dobrą praktyka jest także wydzielenie danych walidacyjnych, jednak dla naszego tutaj prostego zadania nie są one wymagane.

In [11]:
train_dataset, test_dataset = dataset.random_split(0.7, seed=1)
train_dataset_df = train_dataset.to_pandas_dataframe()
test_dataset_df = test_dataset.to_pandas_dataframe()

print(train_dataset_df.describe())

       petal_length  petal_width  sepal_length  sepal_width
count    110.000000   110.000000    110.000000   110.000000
mean       5.830000     3.016364      3.740909     1.178182
std        0.792343     0.424594      1.694367     0.725724
min        4.300000     2.000000      1.100000     0.100000
25%        5.100000     2.800000      1.600000     0.325000
50%        5.750000     3.000000      4.300000     1.300000
75%        6.400000     3.200000      5.000000     1.800000
max        7.700000     4.100000      6.700000     2.400000


Dalej potrzebna będzie nam nazwa stworzonego klastra, na którym będziemy obliczać nasze modele. Aby się nie pomylić wyświetlimy dostępne klastry na naszym zasobie.

In [6]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget

ComputeTarget.list(workspace)

[AmlCompute(workspace=Workspace.create(name='chmury-aml', subscription_id='03d60a7f-8a1f-4aa2-8e20-3f45e41abc75', resource_group='aml-classification'), name=cpu-cluster-chm, id=/subscriptions/03d60a7f-8a1f-4aa2-8e20-3f45e41abc75/resourceGroups/aml-classification/providers/Microsoft.MachineLearningServices/workspaces/chmury-aml/computes/cpu-cluster-chm, type=AmlCompute, provisioning_state=Succeeded, location=eastus, tags=None),
 {
   "id": "/subscriptions/03d60a7f-8a1f-4aa2-8e20-3f45e41abc75/resourceGroups/aml-classification/providers/Microsoft.MachineLearningServices/workspaces/chmury-aml/computes/cpu-instances",
   "name": "cpu-instances",
   "location": "eastus",
   "tags": null,
   "properties": {
     "description": null,
     "computeType": "ComputeInstance",
     "computeLocation": "eastus",
     "resourceId": null,
     "provisioningErrors": null,
     "provisioningState": "Succeeded",
     "properties": {
       "vmSize": "STANDARD_DS2_V2",
       "applications": [
         {
 

W tym kroku zaczniemy podłączać się do naszego klastra oraz zabezpieczymy się na wypadek gdyby to się nie udało. Nasze zadanie możemy wykonywać tylko na typie klastra AmlCompute.

In [12]:
# Choose a name for your cluster.
amlcompute_cluster_name = "cpu-cluster-chm"

found = False
# Check if this compute target already exists in the workspace.
cts = workspace.compute_targets

if amlcompute_cluster_name in cts and cts[amlcompute_cluster_name].type == 'AmlCompute':
    found = True
    print('Found existing training cluster.')
     # Get existing cluster
     # Method 1:
    aml_remote_compute = cts[amlcompute_cluster_name]
     # Method 2:
     # aml_remote_compute = ComputeTarget(ws, amlcompute_cluster_name)
    
    print('Checking cluster status...')
    # Can poll for a minimum number of nodes and for a specific timeout.
    # If no min_node_count is provided, it will use the scale settings for the cluster.
    aml_remote_compute.wait_for_completion(show_output = True, min_node_count = 0, timeout_in_minutes = 20)

if not found:
    print('Not found cluster...')

Found existing training cluster.
Checking cluster status...
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


Wyświetlamy metryki, których można użyć podczas uczenia przy problemie klasyfikacji do optymalizacji. 

In [8]:
from azureml.train import automl

# List of possible primary metrics is here:
# https://docs.microsoft.com/en-us/azure/machine-learning/how-to-configure-auto-train#primary-metric
    
# Get a list of valid metrics for your given task
automl.utilities.get_primary_metrics('classification')

['norm_macro_recall',
 'accuracy',
 'AUC_weighted',
 'average_precision_score_weighted',
 'precision_score_weighted']

Powszechnie najlepsza metryką jest AUC_weighted, która bazuje na liczbie pozytywnie i negatywnie sklasyfikowanych obiektów i ją użyjemy w tym projekcie.

Musimy następnie stworzyć podwaliny naszego projektu i określić wszystkie wczesniejsze parametry oraz dodać kilka nowych dotyczących uczenia. Stworzymy eksperyment, w którym dobierzemy pewne parametry do naszego zbioru

In [13]:
import logging
import os

from azureml.train.automl import AutoMLConfig

project_folder = './automl'
os.makedirs(project_folder, exist_ok=True)

automl_config = AutoMLConfig(compute_target=aml_remote_compute,
                             task='classification',
                             primary_metric='AUC_weighted',
                             experiment_timeout_minutes=15,                            
                             training_data=train_dataset,
                             label_column_name="label",
                             n_cross_validations=5,
                             # blacklist_models='XGBoostClassifier', 
                             # iteration_timeout_minutes=5,                                                    
                             enable_early_stopping=True,
                             featurization='auto',
                             debug_log='automated_ml_errors.log',
                             verbosity=logging.INFO,
                             path=project_folder
                             # **automl_settings
                             )

Teraz najważniejsza część programu, czyli uruchomienie naszego eksperymentu. Jak każdy ważny eksperyment należy go porządnie nazwać, a następnie uruchomić i poczekać aż nasze zadanie zrobi całą pracę. Przy nazwie eksperymentu warto przemyśleć, aby uzależnić ją od pewnej zmieniającej się ale nie powtarzającej się danej. Jeżeli tego nie zrobimy eksperyment będzie się nadpisywać w jednym pliku, czyli utracimy cenne dane, które możemy potem przeanalizować. 

In [14]:
from azureml.core import Experiment
from datetime import datetime

now = datetime.now()
time_string = now.strftime("%m-%d-%Y-%H")
experiment_name = "classif-find-species-{0}".format(time_string)
print(experiment_name)

experiment = Experiment(workspace=workspace, name=experiment_name)

import time
start_time = time.time()
            
run = experiment.submit(automl_config, show_output=True)

print('Manual run timing: --- %s seconds needed for running the whole Remote AutoML Experiment ---' % (time.time() - start_time))

classif-find-species-01-27-2021-01
Running on remote.
No run_configuration provided, running on cpu-cluster-chm with default configuration
Running on remote compute: cpu-cluster-chm
Parent Run ID: AutoML_908933ad-fabd-4e6f-a3f9-3f39d1df9df7

Current status: FeaturesGeneration. Generating features for the dataset.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              

W następnych krokach będziemy sprawdzać dane statystyczne naszego modelu. Skorzystamy z widgetów, które nam to udostepnią. Poczatkowo Zobaczymy ile czasu uczył się nasz model - tutaj około 23 minut. Następnie wydzielamy najlepszy model, który udało nam się uzyskać w eksperymencie i będziemy badać jego dokładność.

In [15]:
from azureml.widgets import RunDetails
RunDetails(run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

In [16]:
import time
import datetime as dt

run_details = run.get_details()

# Like: 2020-01-12T23:11:56.292703Z
end_time_utc_str = run_details['endTimeUtc'].split(".")[0]
start_time_utc_str = run_details['startTimeUtc'].split(".")[0]
timestamp_end = time.mktime(datetime.strptime(end_time_utc_str, "%Y-%m-%dT%H:%M:%S").timetuple())
timestamp_start = time.mktime(datetime.strptime(start_time_utc_str, "%Y-%m-%dT%H:%M:%S").timetuple())

parent_run_time = timestamp_end - timestamp_start
print('Run Timing: --- %s seconds needed for running the whole Remote AutoML Experiment ---' % (parent_run_time))

Run Timing: --- 1345.0 seconds needed for running the whole Remote AutoML Experiment ---


Sprawdzenie, wydzielenie najlepszego uzyskanego modelu.

In [17]:
best_run, fitted_model = run.get_output()
print(best_run)
print(fitted_model)

Run(Experiment: classif-find-species-01-27-2021-01,
Id: AutoML_908933ad-fabd-4e6f-a3f9-3f39d1df9df7_11,
Type: azureml.scriptrun,
Status: Completed)
Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                                 feature_sweeping_config=None,
                                 feature_sweeping_timeout=None,
                                 featurization_config=None, force_text_dnn=None,
                                 is_cross_validation=None,
                                 is_onnx_compatible=None, logger=None,
                                 observer=None, task=None, working_dir=None)),
                ('RobustScaler',
                 RobustScaler(copy...
                 ExtraTreesClassifier(bootstrap=True, ccp_alpha=0.0,
                                      class_weight=None, criterion='entropy',
                                      max_depth=None, max_features=0.5,
       

Teraz czas na testy. Z testowej tabeli należy usunąć dane, które mówią nam jaki to gatunek, czyli musimy usunąć kolumną label z danych testowych.

In [20]:
import pandas as pd

#Remove Label/y column
if 'label' in test_dataset_df.columns:
    y_test_df = test_dataset_df.pop('label')

x_test_df = test_dataset_df

Następnie sprawdzamy jak nasz wytrenowany model poradzi sobie z testowymi danymi. Wyświetlamy 10 pierwszych wygenerowanych gatunków, które nasz algorytm sklasyfikował i myśli, że są poprawne. Łącznie przetestował 40 próbek.

In [21]:
# Try the best model
y_predictions = fitted_model.predict(x_test_df)

print('10 predictions: ')
print(y_predictions[:10])

10 predictions: 
['Iris-setosa' 'Iris-setosa' 'Iris-setosa' 'Iris-setosa' 'Iris-setosa'
 'Iris-setosa' 'Iris-setosa' 'Iris-setosa' 'Iris-setosa' 'Iris-setosa']


In [22]:
y_predictions.shape

(40,)

Na samym końcu chcemy się dowiedzieć jak dobrze nasz model został wyuczony, odliczamy więc jego dokładność, która jak wynik pokazuje wynosi 97%. Jest to bardzo zadowalający wynik.

In [23]:
from sklearn.metrics import accuracy_score

print('label:')
accuracy_score(y_test_df, y_predictions)

label:


0.975

Na koncu odbieramy 5 sekundową pochwałę wzrokową szefa ;).